* Выполняемое задание -- Задание 3 "Многоклассовая классификация и множественная классификация/регрессия"
* Студент: Шеверев Сергей Вячеславови, 22М-05ММ
* Все пункты обязательной части задания

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# устанавливаем точность чисел с плавающей точкой
%precision %.4f

import warnings
warnings.filterwarnings('ignore')  # отключаем предупреждения

## Dataset
Считаем датасет стоимости автомобилей из Задания 1. Датасет модифицирован, в нем выполнены преобразования данных, описанные в блоткноте задания 1.

In [2]:
df = pd.read_csv('car_price_modified.csv')
df.head()

,id,symboling,drivewheel,enginelocation,enginesize,horsepower,price,manufacturer,mpg,coupe,size
0,0,0.0,2,0,1,111,13495.0,14,24.0,1,5.280199
1,1,0.0,2,0,1,111,16500.0,14,24.0,1,5.280199
2,2,2.0,2,0,2,154,16500.0,14,22.5,1,5.875926
3,3,1.0,1,0,1,102,13950.0,17,27.0,0,6.348170
4,4,1.0,3,0,1,115,17450.0,17,20.0,0,6.367348


Поле "id" --- есть индекс. Избавимся от него

In [3]:
df.drop("id", axis=1, inplace=True)
df

,symboling,drivewheel,enginelocation,enginesize,horsepower,price,manufacturer,mpg,coupe,size
0,0.0,2,0,1,111,13495.0,14,24.0,1,5.280199
1,0.0,2,0,1,111,16500.0,14,24.0,1,5.280199
2,2.0,2,0,2,154,16500.0,14,22.5,1,5.875926
3,1.0,1,0,1,102,13950.0,17,27.0,0,6.348170
4,1.0,3,0,1,115,17450.0,17,20.0,0,6.367348
...,...,...,...,...,...,...,...,...,...,...
174,4.0,2,0,1,114,16845.0,16,25.5,0,7.219618
175,4.0,2,0,1,160,19045.0,16,22.0,0,7.209139
176,4.0,2,0,2,134,21485.0,16,20.5,0,7.219618
177,4.0,2,0,1,106,22470.0,16,26.5,0,7.219618


Признаки:
* symboling -- класс безопасности авто (0 - 5)
* driverwheel описывает, какой привод у авто (полный, задний, передний)
* enginelocation --- расположение двигателя (заднее, переднее)
* enginesize --- класс автомобиля по размеру двигателя (малолитражка, средняя, с большим объемом двигателя)
* horsepower --- количество лошадиных сил 
* price --- цена авто
* manufacturer --- порядковый признак производителя авто
* mpg --- усредненный расход топлива автомобиля
* coupe описывает имеет ли авто кузов купе
* size --- геометрические размеры автомобиля


In [4]:
pd.DataFrame(data=['chevrolet',  'honda',  'dodge',  'plymouth',  'subaru',  'isuzu',  'mitsubishi',   'renault',  'toyota',   'volkswagen',  'mazda',  'nissan',  'saab',  'peugeot','alfa-romero',  'mercury',  'volvo',  'audi',  'bmw',  'porsche',  'buick',  'jaguar'])


,0
0,chevrolet
1,honda
2,dodge
3,plymouth
4,subaru
5,isuzu
6,mitsubishi
7,renault
8,toyota
9,volkswagen


Выполним one-hot encoding:

In [5]:
cat_features = ['drivewheel', 'enginesize', 'symboling']
df = pd.get_dummies(df, prefix=cat_features, columns=cat_features)

Для решения задачи "Multiclass classification" требуется наличие более чем двух классов. Поэтому преобразуем количественный признак "price" в категориальный:

In [6]:
df.price.describe()


count      179.000000
mean     13522.732777
std       7871.571293
min       5118.000000
25%       7898.000000
50%      10945.000000
75%      16662.500000
max      45400.000000
Name: price, dtype: float64

In [7]:
to_apply = lambda x: 0 if x <= 7898 else 1 if x  <= 10945 else  2 if x <= 16662.5 else 3
df.price = df.price.apply(to_apply)
df.groupby(by="price").price.count()

price
0    46
1    44
2    44
3    45
Name: price, dtype: int64

Отлично получилось, автомоболей каждого ценового сегмета примерно одинаковое количество.

## Подготовка данных

Разделим данные на тестовую и обучающую выборку, для воспроизводимости результатов зададим random_state, равным 5

In [8]:
from sklearn.model_selection import train_test_split

y = df["price"]
X = df.drop(["price"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

Выполним масштабирование данных:

In [9]:
from sklearn.preprocessing import StandardScaler
features = list(df.keys())
features.remove("price")
numeric_features = ['horsepower', 'manufacturer', 'mpg', 'size']
binary_features = features;
for i in numeric_features:
    binary_features.remove(i)

scaler = StandardScaler()  # воспользуемся стандартным трансформером

# масштабируем обучающую выборку
X_train_scaled = scaler.fit_transform(X_train[numeric_features])

# масштабируем тестовую выборку
X_test_scaled = scaler.transform(X_test[numeric_features])

X_train = np.c_[X_train_scaled, np.array(X_train[binary_features])]
X_test = np.c_[X_test_scaled, np.array(X_test[binary_features])]

## Обучение моделей

Используя кросс-валидацию и подбор гиперпараметров обучим модели (logistic regression, svm, knn, naive bayes, decision tree) с использованием разных стратегий:
* OneVsRest
* OneVsOne
* OutputCode

In [13]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier, OutputCodeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import label_binarize
import math
from time import process_time

labels = sorted(df.price.unique())
models = [LogisticRegression(), KNeighborsClassifier(), SVC(probability=True), GaussianNB(), DecisionTreeClassifier()]
models_name = ['Logistic regression', 'knn', 'svm', 'naive bayes', 'decision tree']
max_name_len = max([len(i) for i in models_name])
models_params = [{'estimator__C': np.linspace(0.1**3, 3, math.floor(3/0.1))},
                 {'estimator__n_neighbors': range (3, 8), 'estimator__weights': ['uniform', 'distance'], 'estimator__p':[1, 2]},
                 {'estimator__C': np.linspace(0.1**3, 3, math.floor(3/0.1)), 'estimator__kernel':['linear', 'poly', 'rbf','sigmoid'], "estimator__degree":range(1, 6), 'estimator__gamma':['scale', 'auto']},
                 {'estimator__var_smoothing': np.logspace(0,-9, num=100)},
                 {"estimator__criterion": ['gini', 'entropy', 'log_loss'], "estimator__splitter": ['best', 'random'], 'estimator__class_weight':[None, 'balanced']}
                ]

OVR = {"best_models_params":[], "accuracies":[], "times": [], "model": []}
OVO = {"best_models_params":[], "accuracies":[], "times": [], "model": []}
OCC = {"best_models_params":[], "accuracies":[], "times": [], "model": []}

for i in range(len(models)):
    if type(models[i]) is not DecisionTreeClassifier:
        time_start = process_time()
        searcher_OVR = GridSearchCV(OneVsRestClassifier(models[i], n_jobs=-1), param_grid = models_params[i], scoring="roc_auc", refit=True,  cv = 5)
        searcher_OVR.fit(X_train, y_train)
        time_stop = process_time()
        OVR["times"].append(time_stop - time_start)
        
        prediction = searcher_OVR.predict_proba(X_test)
        OVR["accuracies"].append(roc_auc_score(y_test, prediction,multi_class='ovr'))
        OVR["best_models_params"].append(searcher_OVR.best_params_)
        OVR["model"].append(models_name[i])
    else:
        time_start = process_time()
        searcher_OVR = GridSearchCV(models[i], param_grid = {"criterion": ['gini', 'entropy', 'log_loss'], "splitter": ['best', 'random'], 'class_weight':[None, 'balanced']}, scoring="roc_auc", refit=True,  cv = 5)
        searcher_OVR.fit(X_train, y_train)
        time_stop = process_time()
        OVR["times"].append(time_stop - time_start)
        
        prediction = searcher_OVR.predict_proba(X_test)
        OVR["accuracies"].append(roc_auc_score(y_test, prediction,multi_class='ovr'))
        OVR["best_models_params"].append(searcher_OVR.best_params_)
        OVR["model"].append(models_name[i])
    
    time_start = process_time()
    searcher_OVO = GridSearchCV(OneVsOneClassifier(models[i], n_jobs=-1), param_grid = models_params[i], scoring="roc_auc", refit=True,  cv = 5)
    time_start = process_time()
    searcher_OVO.fit(X_train, y_train)
    time_stop = process_time()
    OVO["times"].append(time_stop - time_start)
    
    prediction = searcher_OVO.predict(X_test)
    prediction = label_binarize(prediction, classes=labels)
    OVO["accuracies"].append(roc_auc_score(y_test, prediction,multi_class='ovo'))
    OVO["best_models_params"].append(searcher_OVO.best_params_)
    OVO["model"].append(models_name[i])
    
    time_start = process_time()
    searcher_OCC = GridSearchCV(OutputCodeClassifier(models[i], n_jobs=-1), param_grid = models_params[i], scoring="roc_auc", refit=True,  cv = 5)
    searcher_OCC.fit(X_train, y_train)
    time_stop = process_time()
    OCC["times"].append(time_stop - time_start)
    
    prediction = searcher_OCC.predict(X_test)
    prediction = label_binarize(prediction, classes=labels)
    OCC["accuracies"].append(roc_auc_score(y_test, prediction,multi_class='ovo'))
    OCC["best_models_params"].append(searcher_OCC.best_params_)
    OCC["model"].append(models_name[i])


## Результаты обучения

Распечатаем таблицы результатов обучения:

In [14]:
for i in zip([OVR, OVO, OCC], ["OneVsRest results", "OneVsOne results", "OutputCode results"]):
    RES = pd.DataFrame(data=i[0])
    RES.drop("best_models_params", axis=1,  inplace=True)
    print(str(i[1]) + f" Total time is {RES.times.sum():.3f}" + f" Total accuracy is {RES.accuracies.sum():.3f}")
    print(RES, end="\n\n")

OneVsRest results Total time is 36.844 Total accuracy is 4.408
   accuracies      times                model
0    0.810036   0.717387  Logistic regression
1    0.898437   0.434589                  knn
2    0.911517  32.964206                  svm
3    0.927295   2.677238          naive bayes
4    0.860744   0.050414        decision tree

OneVsOne results Total time is 39.314 Total accuracy is 3.529
   accuracies      times                model
0    0.583333   0.688348  Logistic regression
1    0.816667   0.489943                  knn
2    0.500000  34.846735                  svm
3    0.814815   2.908060          naive bayes
4    0.813889   0.381222        decision tree

OutputCode results Total time is 44.027 Total accuracy is 3.363
   accuracies      times                model
0    0.500000   0.728340  Logistic regression
1    0.816667   0.488037                  knn
2    0.500000  39.474222                  svm
3    0.731481   2.845831          naive bayes
4    0.814815   0.490350   

### Скорость:

1) OneVsOne -- быстрее всех обучается
2) OneVsRest -- незначительно медленнее чем OneVsOne (на уровне погрешности измерения)
3) OutputCode --  значительно медленнее других стратегий (примерно на 5 секунд процессорного времени)

### Точность:
1) OneVsRest -- показывает ощутимо большую точность, нежели другие стратегии
2) OutputCode -- несколько точнее OneVsOne, но незначительно
3) OneVsOne -- обладает худшей точностью

* Модели knn, decision tree и naive bayes обладают высокими показателями точности при применении всех стратегий.
